# Models
> Data models for correlation and logging context.

In [ ]:
#| default_exp logs.logs

In [4]:
#| export
from dc_logger.logs.tracing import Correlation, Entity, CorrelationManager, MultiTenant, Entity, HTTPDetails

from dc_logger.client.base import LogLevel

In [7]:
#| exporti

import datetime as dt
from typing import Optional, Dict, Any, Callable
from dataclasses import dataclass, field
import json


In [ ]:
#| export

@dataclass
class Log:
    """Enhanced log entry with structured JSON format"""

    timestamp: str
    level: LogLevel
    logger: str
    message: str
    user: Optional[str] = None
    action: Optional[str] = None
    entity: Optional[Entity] = None
    status: str = "info"
    duration_ms: Optional[int] = None
    correlation: Optional[Correlation] = None
    multi_tenant: Optional[MultiTenant] = None
    http_details: Optional[HTTPDetails] = None
    extra: Dict[str, Any] = field(default_factory=dict)

    def to_dict(self) -> Dict[str, Any]:
        result = {
            "timestamp": self.timestamp,
            "level": self.level.value,
            "logger": self.logger,
            "message": self.message,
            "user": self.user
            or (
                self.multi_tenant.user_id
                if self.multi_tenant and self.multi_tenant.user_id
                else None
            ),
            "action": self.action,
            "status": self.status,
            "duration_ms": self.duration_ms,
            "entity": self.entity.to_dict() if self.entity else None,
            "correlation": self.correlation.__dict__ if self.correlation else None,
            "multi_tenant": self.multi_tenant.__dict__ if self.multi_tenant else None,
            "http_details": self._serialize_http_details(),
            "extra": self.extra,
        }
        return {k: v for k, v in result.items() if v is not None}

    def to_json(self) -> str:
        return json.dumps(self.to_dict(), default=str)

    @classmethod
    def create(cls, level: LogLevel, message: str, logger: str, **kwargs) -> "Log":
        timestamp = dt.datetime.now().isoformat() + "Z"
        user = kwargs.get("user")
        action = kwargs.get("action")
        status = kwargs.get("status", "info")
        duration_ms = kwargs.get("duration_ms")
        extra = kwargs.get("extra", {})

        entity_obj = Entity.from_any(kwargs.get("entity"))
        correlation_obj = Correlation(**kwargs.get("correlation", {})) if kwargs.get("correlation") else None
        multi_tenant_obj = MultiTenant.from_kwargs(kwargs, user)
        http_details_obj = HTTPDetails.from_kwargs(kwargs)

        if not user and multi_tenant_obj and multi_tenant_obj.user_id:
            user = multi_tenant_obj.user_id

        return cls(
            timestamp=timestamp,
            level=level,
            logger=logger,
            message=message,
            user=user,
            action=action,
            entity=entity_obj,
            status=status,
            duration_ms=duration_ms,
            correlation=correlation_obj,
            multi_tenant=multi_tenant_obj,
            http_details=http_details_obj,
            extra=extra,
        )

    def _serialize_http_details(self) -> Optional[Dict[str, Any]]:
        if not self.http_details:
            return None
        http_details_dict = {}
        if self.http_details.method:
            http_details_dict["method"] = self.http_details.method
        if self.http_details.url:
            http_details_dict["url"] = self.http_details.url
        if self.http_details.headers:
            safe_headers = {}
            for k, v in self.http_details.headers.items():
                if k.lower() not in [
                    "authorization",
                    "cookie",
                    "x-domo-authentication",
                ]:
                    safe_headers[k] = v
            if safe_headers:
                http_details_dict["headers"] = safe_headers
        if self.http_details.params:
            http_details_dict["params"] = self.http_details.params
        if self.http_details.request_body:
            if (
                isinstance(self.http_details.request_body, str)
                and len(self.http_details.request_body) > 500
            ):
                http_details_dict["request_body"] = (
                    self.http_details.request_body[:500] + "..."
                )
            else:
                http_details_dict["request_body"] = self.http_details.request_body
        if self.http_details.response_body:
            if (
                isinstance(self.http_details.response_body, str)
                and len(self.http_details.response_body) > 500
            ):
                http_details_dict["response_body"] = (
                    self.http_details.response_body[:500] + "..."
                )
            else:
                http_details_dict["response_body"] = self.http_details.response_body
        return http_details_dict if http_details_dict else None


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export('./models.ipynb')

In [ ]:
#| export
